# 🏥 Costream: Time-Series Anomaly Detection Demo (ECG)

This notebook demonstrates how to use the **`costream`** package for **Univariate Anomaly Detection**.

While `costream` was designed for fall detection, its core logic (Cost-Sensitive Classification + Streaming Segmentation) applies perfectly to detecting medical anomalies, such as heart arrhythmias.

**Dataset**: `ECGDiffCount3` (via `aeon`)
**Goal**: Detect abnormal heartbeats in a continuous stream, prioritizing Recall (don't miss an event).

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# --- Costream Imports ---
from costream.segmentation.training_segmenter import create_training_data
from costream.model.cost_classifier_cv import CostClassifierCV
from costream.evaluation.tester import run_experiment, ModelSpec
from costream.evaluation.visualization import plot_confidence, set_style

set_style("whitegrid")

## 1. Data Loading (Aeon)

We load the `ECGDiffCount3` dataset. We ensure it is flattened to 1D (Univariate) and labels are mapped to `0` (Normal) vs `1` (Anomaly).

In [ ]:
try:
    # Try the specific loader requested
    from aeon.datasets import load_ecg_diff_count_3
    # Note: Adjust unpacking based on actual return signature of your specific loader version
    # Assuming: X, y split or single return. We use a generic fallback below if this varies.
    X_test, y_test, X_train, y_train = load_ecg_diff_count_3("supervised")
except ImportError:
    # Fallback to standard AEON API if specific function name differs
    print("⚠️ Specific loader not found, using generic load_classification...")
    from aeon.datasets import load_classification
    X_train, y_train = load_classification("ECGDiffCount3", split="train")
    X_test, y_test = load_classification("ECGDiffCount3", split="test")

# 1. Adapt Dimensions: (N, 1, T) -> (N, T)
if X_train.ndim == 3:
    X_train = X_train.squeeze()
if X_test.ndim == 3:
    X_test = X_test.squeeze()

# 2. Adapt Labels to Binary (0=Normal, 1=Anomaly)
# We assume the minority class is the Anomaly. Let's check counts.
classes, counts = np.unique(y_train, return_counts=True)
minority_class = classes[np.argmin(counts)]

print(f"Original Classes: {classes}, Counts: {counts}")
print(f"Treating Class '{minority_class}' as Anomaly (1)")

y_train_bin = (y_train == minority_class).astype(int)
y_test_bin = (y_test == minority_class).astype(int)

print(f"Train Shape: {X_train.shape} | Anomalies: {sum(y_train_bin)}")
print(f"Test Shape:  {X_test.shape} | Anomalies: {sum(y_test_bin)}")

## 2. Prepare for Costream

`Costream` data loaders typically expect a **List of DataFrames**, where each DataFrame represents one recording (one subject/trial). 

We will convert the numpy arrays into this format to utilize the full pipeline.

In [ ]:
def to_dataframe_list(X, y):
    dfs = []
    for signal, label in zip(X, y):
        # Create DF with 'ecg' column
        df = pd.DataFrame(signal, columns=['ecg'])
        # The label applies to the whole segment in this dataset context,
        # but for the segmenter, we mark the 'event' at the center if it's an anomaly.
        df['label'] = 0
        if label == 1:
            mid_point = len(signal) // 2
            df.loc[mid_point, 'label'] = 1
        dfs.append(df)
    return dfs

train_dfs = to_dataframe_list(X_train, y_train_bin)

# Visualize one Anomaly
anomaly_idx = np.where(y_train_bin == 1)[0][0]
plt.figure(figsize=(10,3))
plt.plot(train_dfs[anomaly_idx]['ecg'], label="Signal")
plt.title(f"Example Anomaly (Train Idx {anomaly_idx})")
plt.legend()
plt.show()

## 3. Training Segmentation

We segment the signals into windows. Since this dataset is already short segments (~100-2000 points), we will set a small window size relative to the signal length.

In [ ]:
# Determine appropriate window size based on dataset length
sig_len = X_train.shape[1]
FREQ = 100 # Assumed Hz for this demo
win_sec = (sig_len / FREQ) * 0.5 # Window = 50% of signal length

print(f"Signal Len: {sig_len}, Using Window Size: {win_sec:.2f}s")

X_train_seg, y_train_seg = create_training_data(
    train_dfs,
    feature_cols=['ecg'],
    window_size=win_sec,
    step=win_sec / 2, # 50% overlap
    freq=FREQ,
    activity_threshold=0.0, # Keep all windows (don't filter quiet parts for ECG)
    spacing="na" # Center the event
)

# Flatten 3D (N, Win, 1) to 2D (N, Win) for sklearn
if X_train_seg.ndim == 3:
    X_train_seg = X_train_seg[:, :, 0]

print(f"Segmented Shape: {X_train_seg.shape}")
print(f"Class Balance: {np.bincount(y_train_seg)}")

## 4. Train Cost-Sensitive Model

We use **`CostClassifierCV`** to penalize missing an anomaly (False Negative) significantly more than raising a False Alarm.

In [ ]:
model = CostClassifierCV(
    base_estimators=[
        LogisticRegression(max_iter=500, solver='liblinear'),
        RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
    ],
    alpha=3.0,          # 1 Missed Anomaly = 3 False Alarms cost-wise
    method="stacking",  # Use meta-learner to fuse probs
    cv=3,
    random_state=42
)

print("🧠 Training Model...")
model.fit(X_train_seg, y_train_seg)
print(f"Done. Threshold: {model.threshold_:.3f}, Weights: {model.weights_}")

## 5. Streaming Evaluation

To test the "streaming" capabilities, we will stitch together the **Test Set** samples into continuous long streams. This simulates a real monitoring scenario where anomalies occur sporadically.

In [ ]:
# Create 3 synthetic streams by concatenating test samples
n_streams = 3
samples_per_stream = 20

test_signals = []
test_events = []

for i in range(n_streams):
    # Select random samples from test set
    indices = np.random.choice(len(X_test), samples_per_stream, replace=False)
    
    # Stitch signal
    stream = np.concatenate(X_test[indices])
    
    # Calculate where the anomalies are in this new stream
    current_idx = 0
    events = []
    for idx in indices:
        if y_test_bin[idx] == 1:
            # Anomaly is at the center of this segment
            events.append(current_idx + (sig_len // 2))
        current_idx += sig_len
        
    test_signals.append(stream)
    test_events.append(events if events else -1)

print(f"Generated {len(test_signals)} streams of length {len(test_signals[0])}.")

In [ ]:
# Run the Evaluation Engine
results = run_experiment(
    X_train_seg, y_train_seg,
    test_signals=test_signals,
    test_event_points=test_events,
    model_specs=[ModelSpec("Cost_Ensemble", model)],
    window_size=win_sec,
    step=win_sec / 2,
    freq=FREQ,
    signal_thresh=0.0, # Process all windows
    tolerance=win_sec, # Allow detection within 1 window length
    verbose=True
)

display(results)

## 6. Visualization

Let's look at one of the stitched streams to see the model's confidence map reacting to the anomalies.

In [ ]:
# Pick a stream that definitely has events
stream_idx = next(i for i, ev in enumerate(test_events) if ev != -1)

signal = test_signals[stream_idx]
events = test_events[stream_idx]

# Re-run inference just for this plot (or grab from cache if extended)
from costream.segmentation.streaming_segmenter import sliding_window_inference
from costream.evaluation.event_detection import evaluate_recording

conf_map, _ = sliding_window_inference(
    signal, model, 
    window_size=win_sec, step=win_sec/2, freq=FREQ, signal_thresh=0.0
)

# Calculate stats for title
cm, highs, _ = evaluate_recording(
    len(signal), events, conf_map, 
    confidence_thresh=model.threshold_, 
    window_size=win_sec, tolerance=win_sec, freq=FREQ
)
tn, fp, fn, tp = cm.ravel()

plot_confidence(
    signal, conf_map, events,
    tp, fp, tn, fn,
    model_name="ECG Cost Ensemble",
    high_conf=highs,
    thresh_line=model.threshold_,
    freq=FREQ
)